In [1]:
import pandas as pd
import glob
import os
import sys
from tqdm import tqdm

In [2]:
path = "C:\\Users\\Henry\\Documents\\data\\2020_04_23\\Trace\\"
all_files = glob.glob(path + "*.csv")
all_files

['C:\\Users\\Henry\\Documents\\data\\2020_04_23\\Trace\\trace_csf.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_04_23\\Trace\\trace_fly_remote.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_04_23\\Trace\\trace_jdbc.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_04_23\\Trace\\trace_local.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_04_23\\Trace\\trace_osb.csv',
 'C:\\Users\\Henry\\Documents\\data\\2020_04_23\\Trace\\trace_remote_process.csv']

In [3]:
dfs = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    dfs.append(df)

In [4]:
for df in dfs:
    if df['callType'].iloc[0]=='JDBC' or df['callType'].iloc[0]=='LOCAL':
        df['serviceName'] = df['dsName']

In [5]:
df = pd.concat(dfs, axis=0, ignore_index=True)
df.head()

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName,dsName
0,CSF,1587571200072,338.0,True,d46b0171a29e10407940,9d6cb171a29e10487945,19874171a29e10407941,os_022,csf_001,NaN
1,CSF,1587571200073,338.0,True,b382e171a29e10417942,e83f5171a29e10497947,195a2171a29e10417943,os_022,csf_001,NaN
2,CSF,1587571200074,484.0,True,f77b1171a29e10434832,749dc171a29e104a4835,ab5e8171a29e10434833,os_022,csf_001,NaN
3,CSF,1587571200089,786.0,True,30f3f171a29e10456054,a5084171a29e10596059,17368171a29e10456055,os_021,csf_001,NaN
4,CSF,1587571200090,784.0,True,c0daf171a29e10486056,fafbc171a29e105a6061,c38fa171a29e10496057,os_021,csf_001,NaN


In [6]:
df = df.drop('dsName', axis=1)
# del dfs

In [7]:
df = df.sort_values(["traceId" ,"startTime"], ignore_index=True)
df = df.head(500000)
df

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName
0,OSB,1587579090784,1540.0,True,00007171a31677604200,b27e5171a31677604201,None,os_022,osb_001
1,CSF,1587579090790,1525.0,True,00007171a31677604200,acdcd171a31677664203,b27e5171a31677604201,os_022,csf_001
2,RemoteProcess,1587579090796,1520.0,True,00007171a31677604200,2e8ba171a316776c9828,acdcd171a31677664203,docker_002,csf_001
3,FlyRemote,1587579090997,5.0,True,00007171a31677604200,23a11171a31678359892,2e8ba171a316776c9828,docker_002,fly_remote_001
4,CSF,1587579091097,347.0,True,00007171a31677604200,76499171a31678989929,2e8ba171a316776c9828,docker_002,csf_002
...,...,...,...,...,...,...,...,...,...
499995,RemoteProcess,1587574478649,45.0,True,0c6e4171a2d0146c8902,de4b5171a2d017397967,87b3a171a2d016d29310,docker_008,csf_003
499996,LOCAL,1587574478652,16.0,True,0c6e4171a2d0146c8902,db041171a2d0173c7971,de4b5171a2d017397967,docker_008,db_003
499997,JDBC,1587574478653,7.0,True,0c6e4171a2d0146c8902,2bd24171a2d0173d7974,db041171a2d0173c7971,docker_008,db_003
499998,JDBC,1587574478661,7.0,True,0c6e4171a2d0146c8902,31143171a2d017457977,db041171a2d0173c7971,docker_008,db_003


In [8]:
df1 = df[df['callType']=='RemoteProcess']
df1 = df1[['pid','cmdb_id']]
df1 = df1.set_index('pid')

In [9]:
csf_dict = df1.to_dict()
csf_cmdb = {str(key):str(values) for key, values in csf_dict['cmdb_id'].items()}

In [17]:
csf_cmdb['acdcd171a31677664203']

'docker_002'

In [16]:
for index, row in tqdm(df.iterrows()):
    if row['id'] in csf_cmdb:
        df.at[index, 'serviceName'] = csf_cmdb[row['id']]

500000it [00:35, 14232.05it/s]


In [18]:
df.head()

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName
0,OSB,1587579090784,1540.0,True,00007171a31677604200,b27e5171a31677604201,None,os_022,osb_001
1,CSF,1587579090790,1525.0,True,00007171a31677604200,acdcd171a31677664203,b27e5171a31677604201,os_022,docker_002
2,RemoteProcess,1587579090796,1520.0,True,00007171a31677604200,2e8ba171a316776c9828,acdcd171a31677664203,docker_002,csf_001
3,FlyRemote,1587579090997,5.0,True,00007171a31677604200,23a11171a31678359892,2e8ba171a316776c9828,docker_002,fly_remote_001
4,CSF,1587579091097,347.0,True,00007171a31677604200,76499171a31678989929,2e8ba171a316776c9828,docker_002,docker_007


In [19]:
elapse_time = {}
children = {}
parent_service = {}
for index, row in tqdm(df.iterrows()):
    if row['pid'] != 'None':
        if row['pid'] in children.keys():
            children[row['pid']].append(row['id'])
        else:
            children[row['pid']] = [row['id']]
    elapse_time [row['id']] = float(row['elapsedTime'])
    # parent_service[row['id']] = str(row['cmdb_id'] + ':' + row['serviceName'])
    parent_service[row['id']] = row['serviceName']

500000it [00:44, 11158.53it/s]


In [20]:
df.head()

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName
0,OSB,1587579090784,1540.0,True,00007171a31677604200,b27e5171a31677604201,None,os_022,osb_001
1,CSF,1587579090790,1525.0,True,00007171a31677604200,acdcd171a31677664203,b27e5171a31677604201,os_022,docker_002
2,RemoteProcess,1587579090796,1520.0,True,00007171a31677604200,2e8ba171a316776c9828,acdcd171a31677664203,docker_002,csf_001
3,FlyRemote,1587579090997,5.0,True,00007171a31677604200,23a11171a31678359892,2e8ba171a316776c9828,docker_002,fly_remote_001
4,CSF,1587579091097,347.0,True,00007171a31677604200,76499171a31678989929,2e8ba171a316776c9828,docker_002,docker_007


In [21]:
df['actual_time'] = 0.0
df['path'] = ''
for index, row in tqdm(df.iterrows()):
    total_child = 0.0
    if row['pid'] not in parent_service.keys():
        df.at[index, 'path'] = 'Start-' + row['serviceName']
    else:
        df.at[index, 'path'] = parent_service[row['pid']] + '-' + row['serviceName']

    if row['id'] not in children.keys():
        df.at[index, 'actual_time'] = row['elapsedTime']
        continue
    for child in children[row['id']]:
        total_child += elapse_time[child]
    df.at[index, 'actual_time'] = row['elapsedTime'] - total_child

500000it [00:54, 9185.34it/s]


In [22]:
df = df.sort_values(["traceId" ,"startTime"], ignore_index=True)

In [23]:
df.head()

,callType,startTime,elapsedTime,success,traceId,id,pid,cmdb_id,serviceName,actual_time,path
0,OSB,1587579090784,1540.0,True,00007171a31677604200,b27e5171a31677604201,None,os_022,osb_001,15.0,Start-osb_001
1,CSF,1587579090790,1525.0,True,00007171a31677604200,acdcd171a31677664203,b27e5171a31677604201,os_022,docker_002,5.0,osb_001-docker_002
2,RemoteProcess,1587579090796,1520.0,True,00007171a31677604200,2e8ba171a316776c9828,acdcd171a31677664203,docker_002,csf_001,569.0,docker_002-csf_001
3,FlyRemote,1587579090997,5.0,True,00007171a31677604200,23a11171a31678359892,2e8ba171a316776c9828,docker_002,fly_remote_001,5.0,csf_001-fly_remote_001
4,CSF,1587579091097,347.0,True,00007171a31677604200,76499171a31678989929,2e8ba171a316776c9828,docker_002,docker_007,297.0,csf_001-docker_007


In [24]:
df.to_csv('processed_data.csv', index=False)